In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from pprint import pprint

df_2012_train = pd.read_csv('df_2012_train.csv') 
df_2012_train = df_2012_train.set_index('fips')
df_2012_test = pd.read_csv('df_2012_test.csv') 
df_2012_test = df_2012_test.set_index('fips')
df_2016_train = pd.read_csv('df_2016_train.csv') 
df_2016_train = df_2016_train.set_index('county_name')
df_2016_test = pd.read_csv('df_2016_test.csv') 
df_2016_test = df_2016_test.set_index('county_name')

In [2]:
df_2012_train.head(2)

,state_abbreviation,county_name,diff_2012,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,perc_2races,...,hc_value_1k,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
fips,,,,,,,,,,,,,,,,,,,,,
50011,VT,Franklin County,0.235485,0.607936,0.554590,0.852168,0.752126,0.454698,0.679059,0.334973,...,0.618064,0.598671,0.573469,0.538418,0.619473,0.426309,0.575188,0.366486,0.414746,0.166732
46093,SD,Meade County,-0.428453,0.800869,0.501383,0.814002,0.806294,0.452165,0.683144,0.451818,...,0.146813,0.044439,0.491513,0.502540,0.659653,0.397049,0.523883,0.330485,0.352493,0.177449


In [2]:
df_2012_train_x = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y = df_2012_train[['diff_2012']]

df_2016_train_x = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y = df_2016_train[['diff_2016']]

df_2012_test_x = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y = df_2012_test[['diff_2012']]

df_2016_test_x = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y = df_2016_test[['diff_2016']]

full_train_x = [df_2012_train_x, df_2016_train_x]
full_train_x = pd.concat(full_train_x)
df_2012_train_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y = [df_2012_train_y, df_2016_train_y]
full_train_y = pd.concat(full_train_y)

full_test_x = [df_2012_test_x, df_2016_test_x]
full_test_x = pd.concat(full_test_x)
df_2012_test_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y = [df_2012_test_y, df_2016_test_y]
full_test_y = pd.concat(full_test_y)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [6]:
#2016 feature selection

rgr = GradientBoostingRegressor(n_estimators=100, loss='ls', learning_rate=0.1, max_depth=3, min_samples_split=3, random_state=34)

featureSelection = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(df_2016_train_x, df_2016_train_y)

selectedFeatures = featureSelection.transform(df_2016_train_x)
#selectedFeatures

fs = df_2016_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

df_2016_train_x = df_2016_train_x[keep]
df_2016_test_x = df_2016_test_x[keep]

C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['perc_white', 'perc_black', 'perc_asian', 'perc_white_not_hisp', 'perc_oo', 'monthly_costs_m', 'monthly_costs_nm', 'perc_bc', 'num_establishments', 'min_num_firms']


In [5]:
rgr = GradientBoostingRegressor(n_estimators=100, loss='ls', learning_rate=0.1, max_depth=3, min_samples_split=3, random_state=34)
featureSelection = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(full_train_x, full_train_y)

selectedFeatures = featureSelection.transform(full_train_x)
#selectedFeatures

fs = full_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

full_train_x = full_train_x[keep]
full_test_x = full_test_x[keep]



C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['perc_white', 'perc_white_not_hisp', 'perc_oo', 'median_oo_value', 'monthly_costs_m', 'monthly_costs_nm', 'perc_bc', 'wholesale_value_1k', 'num_establishments', 'annual_payroll']


In [10]:
gb = GradientBoostingRegressor(random_state = 33)
pprint(gb.get_params())

{'alpha': 0.9,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'auto',
 'random_state': 33,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}


In [13]:
from sklearn.model_selection import RandomizedSearchCV

# GBmodel = GradientBoostingRegressor()
# param_dist = {"learning_rate": np.linspace(0.05, 0.15,5),
#               "max_depth": range(3, 5),
#               "min_samples_leaf": range(3, 5)}


learning_rate = [2, 1, 0.5, 0.25, 0.1, 0.05, 0.01]
# Number of trees in  forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
# Create the random grid
random_grid = {'learning_rate': learning_rate,
               'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,}
pprint(random_grid)

{'learning_rate': [2, 1, 0.5, 0.25, 0.1, 0.05, 0.01],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [10, 231, 452, 673, 894, 1115, 1336, 1557, 1778, 2000]}


In [15]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
gb = GradientBoostingRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
gb_random = RandomizedSearchCV(estimator = gb, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=31, n_jobs = -1)
# Fit the random search model
gb_random.fit(df_2012_train_x, df_2012_train_y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.5min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=-1,
          param_distributions={'learning_rate': [2, 1, 0.5, 0.25, 0.1, 0.05, 0.01], 'n_estimators': [10, 231, 452, 673, 894, 1115, 1336, 1557, 1778, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
          pre_dispatch='2*n_jobs', random_state=31, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [16]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    print("rmse:",rmse)
    r2 = explained_variance_score(test_labels, predictions)
    print("r2:",r2)

print("BASE MODEL")
base_model = GradientBoostingRegressor(random_state = 31)
base_model.fit(df_2012_train_x, df_2012_train_y)
base_accuracy = evaluate(base_model, df_2012_test_x, df_2012_test_y)

BASE MODEL


C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


rmse: 0.1479730731564178
r2: 0.7395848717727767


In [17]:
gb_random.best_params_

{'n_estimators': 894,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 20,
 'learning_rate': 0.05}

In [18]:
print('best random')
best_random =gb_random.best_estimator_
random_accuracy = evaluate(best_random, df_2012_test_x, df_2012_test_y)

best random
rmse: 0.139825459547378
r2: 0.7675866616770286


In [7]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [10, 20, 30],
    'max_features': ['sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [800, 900, 1000],
    'learning_rate': [0.03, 0.05, 0.07, .1]}
# Create a based model
gb = GradientBoostingRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = gb, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [20]:
# Fit the grid search to the data
grid_search.fit(df_2012_train_x, df_2012_train_y)
grid_search.best_params_



Fitting 3 folds for each of 324 candidates, totalling 972 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 20.8min
[Parallel(n_jobs=-1)]: Done 972 out of 972 | elapsed: 26.6min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.03,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 3,
 'n_estimators': 1000}

In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    print("mse:", mse)
    print("rmse:",rmse)
    mae = mean_absolute_error(test_labels, predictions)
    print("mae:", mae)
    r2 = explained_variance_score(test_labels, predictions)
    print("r2:",r2)

In [22]:
df_2012_train_x2 = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y2 = df_2012_train[['diff_2012']]

df_2016_train_x2 = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y2 = df_2016_train[['diff_2016']]

df_2012_test_x2 = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y2 = df_2012_test[['diff_2012']]

df_2016_test_x2 = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y2 = df_2016_test[['diff_2016']]

full_train_x2 = [df_2012_train_x2, df_2016_train_x2]
full_train_x2 = pd.concat(full_train_x2)
df_2012_train_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y2 = [df_2012_train_y2, df_2016_train_y2]
full_train_y2 = pd.concat(full_train_y2)

full_test_x2 = [df_2012_test_x2, df_2016_test_x2]
full_test_x2 = pd.concat(full_test_x2)
df_2012_test_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y2 = [df_2012_test_y2, df_2016_test_y2]
full_test_y2 = pd.concat(full_test_y2)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [23]:
fts_2012 = ['population', 'perc_o18', 'perc_white', 'perc_white_not_hisp', 'perc_oo', 'median_oo_value', 'monthly_costs_nm', 'num_nonemployers', 'man_value_1k', 'wholesale_value_1k', 'num_establishments']
test_x = df_2016_test_x2[fts_2012]
full_x = full_test_x2[fts_2012]



print('2012 GRID Search on 2012')
grid_accuracy = evaluate(grid_search, df_2012_test_x, df_2012_test_y)
print('2012 GRID Search on 2016')
grid_accuracy = evaluate(grid_search, test_x, df_2016_test_y2)
print('2012 GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_x, full_test_y2)

2012 GRID Search on 2012
mse: 0.018751751149209118
rmse: 0.13693703351982298
mae: 0.10440488952126302
r2: 0.777035171643355
2012 GRID Search on 2016
mse: 0.025626639967522135
rmse: 0.16008322825181323
mae: 0.1241719748388757
r2: 0.7265101894678216
2012 GRID Search on Full
mse: 0.022189195558365626
rmse: 0.14896038251281993
mae: 0.11428843218006934
r2: 0.7572473572066064


In [24]:
grid_search.best_params_

{'learning_rate': 0.03,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 3,
 'n_estimators': 1000}

In [25]:
fts = ['population', 'perc_o18', 'perc_white', 'perc_black', 'perc_white_not_hisp', 'perc_oo', 'monthly_costs_m', 'monthly_costs_nm', 'num_nonemployers', 'man_value_1k', 'wholesale_value_1k', 'num_establishments', 'annual_payroll', 'min_num_firms']
list(zip(fts,list(grid_search.best_estimator_.feature_importances_)))

[('population', 0.08654099822245524),
 ('perc_o18', 0.06424651052296114),
 ('perc_white', 0.1568861822782711),
 ('perc_black', 0.18870443995846511),
 ('perc_white_not_hisp', 0.06265020289564498),
 ('perc_oo', 0.055785384369138435),
 ('monthly_costs_m', 0.1426076534581597),
 ('monthly_costs_nm', 0.06056906079462832),
 ('num_nonemployers', 0.06371823482154422),
 ('man_value_1k', 0.06092402550093417),
 ('wholesale_value_1k', 0.05736730717779741)]

In [26]:
# Fit the grid search to the data
grid_search.fit(df_2016_train_x, df_2016_train_y)
grid_search.best_params_


Fitting 3 folds for each of 324 candidates, totalling 972 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 18.9min
[Parallel(n_jobs=-1)]: Done 972 out of 972 | elapsed: 24.3min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.03,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 4,
 'n_estimators': 800}

In [27]:
fts_2016 = ['perc_white', 'perc_black', 'perc_asian', 'perc_white_not_hisp', 'perc_oo', 'monthly_costs_m', 'monthly_costs_nm', 'perc_bc', 'num_establishments', 'min_num_firms']

test_x = df_2012_test_x2[fts_2016]
full_x = full_test_x2[fts_2016]

print('2016 GRID Search on 2012')
grid_accuracy = evaluate(grid_search, test_x, df_2012_test_y2)
print('2016 GRID Search on 2016')
grid_accuracy = evaluate(grid_search, df_2016_test_x, df_2016_test_y)
print('2016 GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_x, full_test_y2)

2016 GRID Search on 2012
mse: 0.03946589344261203
rmse: 0.19866024625629566
mae: 0.15624946704843984
r2: 0.5305136903688652
2016 GRID Search on 2016
mse: 0.022112566661992527
rmse: 0.14870294772462492
mae: 0.11398606336891468
r2: 0.7592729890083473
2016 GRID Search on Full
mse: 0.030789230052302274
rmse: 0.17546860132884823
mae: 0.13511776520867724
r2: 0.6606319049455618


In [28]:
grid_search.best_params_

{'learning_rate': 0.03,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 4,
 'n_estimators': 800}

In [30]:
list(zip(fts_2016,list(grid_search.best_estimator_.feature_importances_)))

[('perc_white', 0.19157841526969183),
 ('perc_black', 0.06279472828127841),
 ('perc_asian', 0.07383929396401541),
 ('perc_white_not_hisp', 0.1599304111403673),
 ('perc_oo', 0.05657675974524884),
 ('monthly_costs_m', 0.10395103418064872),
 ('monthly_costs_nm', 0.0809670841061856),
 ('perc_bc', 0.11136455538795222),
 ('num_establishments', 0.08430967562377968),
 ('min_num_firms', 0.07468804230083198)]

In [11]:
# Fit the grid search to the data
grid_search.fit(full_train_x, full_train_y)
grid_search.best_params_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   26.5s finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.03,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 900}

In [14]:
predictions = grid_search.predict(full_test_x)

In [16]:
len(predictions)

1246

In [17]:
len(full_test_y)

1246

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
actuals = np.array(full_test_y.diff)
#plt.plot(actuals)
actuals

array(<bound method DataFrame.diff of                         diff
36099               0.092279
37011              -0.502416
8067                0.088112
56029              -0.573213
45067               0.298253
36075               0.078526
42045               0.218194
31099              -0.500634
42009              -0.549291
46053              -0.422129
49057              -0.460341
8053               -0.207358
42035              -0.113559
41009               0.051421
21193              -0.585254
30065              -0.556736
51023              -0.385655
12069              -0.171404
42113              -0.283231
47135              -0.223579
35061               0.024777
27015              -0.166258
48253              -0.545307
39151              -0.003646
28127              -0.220508
8095               -0.463135
1131                0.487399
55087              -0.018347
12033              -0.205599
42119              -0.234756
...                      ...
Atchison County    -0.557473
Lynn 

In [23]:
full_test_y

,diff
36099,0.092279
37011,-0.502416
8067,0.088112
56029,-0.573213
45067,0.298253
36075,0.078526
42045,0.218194
31099,-0.500634
42009,-0.549291
46053,-0.422129


In [54]:
fts_full = ['perc_white', 'perc_white_not_hisp', 'perc_oo', 'median_oo_value', 'monthly_costs_m', 'monthly_costs_nm', 'perc_bc', 'wholesale_value_1k', 'num_establishments', 'annual_payroll']

test_x = df_2012_test_x2[fts_full]
test2_x = df_2016_test_x2[fts_full]

print('FULL GRID Search on 2012')
grid_accuracy = evaluate(grid_search, test_x, df_2012_test_y2)
print('FULL GRID Search on 2016')
grid_accuracy = evaluate(grid_search, test2_x, df_2016_test_y2)
print('FULL GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_test_x, full_test_y)

FULL GRID Search on 2012
mse: 0.031919790509355596
rmse: 0.17866110519459907
mae: 0.13741149221996057
r2: 0.6204370748488228
FULL GRID Search on 2016
mse: 0.023150139867768274
rmse: 0.15215170018034066
mae: 0.11559100224600931
r2: 0.7478065538378071
FULL GRID Search on Full
mse: 0.027534965188561935
rmse: 0.1659366300385841
mae: 0.12650124723298495
r2: 0.6965088840913378


In [55]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 900}

In [10]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [100],
    'max_features': ['sqrt'],
    'min_samples_leaf': [2],
    'min_samples_split': [3],
    'n_estimators': [900],
    'learning_rate': [0.03]}
# Create a based model
gb = GradientBoostingRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = gb, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [56]:
list(zip(lst,list(grid_search.best_estimator_.feature_importances_)))

[('population', 0.07506219888359449),
 ('perc_white', 0.15339720069221613),
 ('perc_black', 0.07028499615253161),
 ('perc_white_not_hisp', 0.1716897021258679),
 ('median_oo_value', 0.06202126553218663),
 ('monthly_costs_m', 0.07258555593753088),
 ('monthly_costs_nm', 0.09739282411537033),
 ('perc_bc', 0.06345892937658154),
 ('num_establishments', 0.07015691427433549),
 ('annual_payroll', 0.08924956676965709),
 ('min_num_firms', 0.07470084614012744)]